# Visualize depthmap (including data augmentation)

In [ ]:
%reload_ext autoreload
%autoreload 2

import datetime
from pathlib import Path
import json
import glob2 as glob
import math
import os
import re
import pickle
import random

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import azureml.core
from azureml.core import Workspace
from azureml.core.dataset import Dataset

from visualization_utils import (IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH, REPO_DIR, 
                                 show_pickle, path_to_ndarray, choose_a_pickle_file, get_datetime)

In [ ]:
pickle_file_path = str(REPO_DIR / "dataset/scans/1583462470-16tvfmb1d0/100/pc_1583462470-16tvfmb1d0_1591122155216_100_000.p")

In [ ]:
qrcode_path = str(REPO_DIR / "dataset/scans/1583462470-16tvfmb1d0/100/")
path_with_wildcard = os.path.join(qrcode_path, "*.p")
list_of_pickle_file_paths = sorted(glob.glob(path_with_wildcard))

In [ ]:
list_of_pickle_file_paths[:5]

## Visualise scan sequence

In [ ]:
# Naive step visualization
i = 20

In [ ]:
# How to use: Execute this cell many times increasing i and seeing how the image changes
print(i, list_of_pickle_file_paths[i])
show_pickle(list_of_pickle_file_paths[i])
i += 1

In [ ]:
# [os.path.basename(str(p)) for p in list_of_pickle_file_paths]

In [ ]:
len(list_of_pickle_file_paths)

In [ ]:
# Problem: although the scan of the child is new and on a different day, the height measurement is the same -> this is weird

In [ ]:
# get_datetime(str(REPO_DIR / "dataset/scans/1583462470-16tvfmb1d0/100/pc_1583462470-16tvfmb1d0_1591427998277_100_005.p"))

In [ ]:
num_height, num_width = 5, 8
fig, axarr = plt.subplots(num_height, num_width, figsize=(25, 18))

for x in range(num_height):
    for y in range(num_width):
        i = x * num_width + y
        fpath = list_of_pickle_file_paths[i]
        dt = get_datetime(fpath)
        arr, targets = path_to_ndarray(fpath)
        axarr[x,y].imshow(arr, cmap='gray', vmin=0, vmax=1); 
        axarr[x,y].set_xticklabels([])
        axarr[x,y].set_yticklabels([])
        
        # axarr[x,y].set_xlabel(f"{dt}")
        
        fname = os.path.basename(fpath)
        axarr[x,y].set_xlabel(fname[-12:]+ f" is {targets[0]:.2f}cm")
        
        # axarr[x,y].set_xlabel(f"is {targets[0]:.2f}cm")
plt.show()

# Display random images from different QRcodes

In [ ]:
scans_dir = REPO_DIR / "dataset/scans/"
qrcodes_with_wildcard = os.listdir(scans_dir)

In [ ]:
choose_a_pickle_file(qrcodes_with_wildcard, scans_dir)

In [ ]:
pickle_file_path = choose_a_pickle_file(qrcodes_with_wildcard, scans_dir)
show_pickle(pickle_file_path)

In [ ]:
num_height, num_width = 5, 8
fig, axarr = plt.subplots(num_height, num_width, figsize=(25, 18))

for x in range(num_height):
    for y in range(num_width):
        pickle_file_path = choose_a_pickle_file(qrcodes_with_wildcard, scans_dir)
        arr, targets = path_to_ndarray(pickle_file_path)
        axarr[x,y].imshow(arr, cmap='gray', vmin=0, vmax=1); 
        axarr[x,y].set_xticklabels([])
        axarr[x,y].set_yticklabels([])
        dt = get_datetime(pickle_file_path)
        axarr[x,y].set_xlabel(f"{targets[0]:.2f}cm {dt}")
plt.show()

# Data Augmentation

In [ ]:
import sys
sys.path.append(str(REPO_DIR / "Models/CNNDepthMap/CNNDepthMap-height/q3-depthmapmultiartifact-plaincnn-height/src"))
import preprocessing

In [ ]:
num_height, num_width = 4, 4
fig, axarr = plt.subplots(num_height, num_width, figsize=(25, 18))

arr, _ = path_to_ndarray(list_of_pickle_file_paths[0])
for x in range(num_height):
    for y in range(num_width):
        arr_augmented = preprocessing.augmentation(arr.reshape(IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH, 1))
        axarr[x,y].imshow(arr_augmented.reshape(IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH), cmap='gray', vmin=0, vmax=1);
        axarr[x,y].set_xticklabels([])
        axarr[x,y].set_yticklabels([])